In [36]:
# Imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.stats import distributions as pdf
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import sklearn.datasets as dset
from sklearn.linear_model import LogisticRegression

In [60]:
# helper functions
def make_data(num_samples, scatter):
    np.random.seed(123)
    class_means =[(1,1),(1,8),(7,1)]
    c1_samples =np.random.multivariate_normal(mean=class_means[0], 
                                             cov=[[scatter[0] *1, 0], [0, scatter[0] *1]], 
                                            size=num_samples[0])
    c2_samples = np.random.multivariate_normal(mean=class_means[1], 
                                            cov=[[scatter[1] *1, 0], [0, scatter[1] *1]], 
                                            size=num_samples[1])
    c3_samples = np.random.multivariate_normal(mean=class_means[2], 
                                            cov=[[scatter[2] *1, 0], [0, scatter[2] *1]], 
                                            size=num_samples[2])
    
    # labels
    c1_labels = np.vstack((np.ones(c1_samples.shape[0]), 
                           np.zeros(c1_samples.shape[0]), 
                           np.zeros(c1_samples.shape[0]))).T
    c2_labels = np.vstack((np.zeros(c2_samples.shape[0]), 
                           np.ones(c2_samples.shape[0]), 
                           np.zeros(c2_samples.shape[0]))).T
    c3_labels = np.vstack((np.zeros(c3_samples.shape[0]), 
                           np.zeros(c3_samples.shape[0]), 
                           np.ones(c3_samples.shape[0]))).T
    
    
    # feature matrix, weight matrix, label matrix (one-hot)
    x = np.vstack((c1_samples, c2_samples, c3_samples))
    x = np.hstack((np.ones(x.shape[0])[:,None],x)) # bias term
    labels = np.vstack((c1_labels, c2_labels, c3_labels))
    
    return x, labels, (c1_samples, c2_samples, c3_samples)

def plot_data(c1_samples,c2_samples,c3_samples):
    ax = plt.subplot()
#     c1_samples,c2_samples,c3_samples=np.log(c1_samples),np.log(c2_samples),np.log(c3_samples)
    
    
    ax.scatter(c1_samples[:,0],c1_samples[:,1])
    ax.scatter(c2_samples[:,0],c2_samples[:,1])
    ax.scatter(c3_samples[:,0],c3_samples[:,1])
    return ax


def loss_fn(x,weights,labels):    
    # scores
    s = x@weights.T  # scores
    eps = np.max(s)  # constant for numerical stability
    probs = (np.exp(s-eps))/ (np.sum(np.exp(s-eps), axis=1)+1e-6)[:,None]
    # Loss function (negative log-likelihood)
    loss = np.sum(-labels*np.log(probs+1e-6))
    dloss_dw = (probs-labels).T@x
    return loss, dloss_dw, probs


def probability(x,weights):
    s = x@weights.T  # scores
    eps = np.max(s)  # constant for numerical stability
    probs = (np.exp(s-eps))/ (np.sum(np.exp(s-eps), axis=1)+1e-6)[:,None]
    return probs


def grid(x,w, mode='sep'):
    grid_resolution = 120
#     x = np.hstack((np.ones(x.shape[0])[:,None],x))
    grid_x = np.linspace(np.min(x[:,1]),np.max(x[:,1]),grid_resolution)
    grid_y = np.linspace(np.min(x[:,2]),np.max(x[:,2]),grid_resolution)

    grid = np.zeros((grid_resolution,grid_resolution))
    for i, ig in enumerate(grid_x):
        for j, jg in enumerate(grid_y):
            feats = np.array((1,ig,jg))[:,None].T
            pr=probability(feats,w)
            if mode == 'sep':
                grid[i,j]=np.argmax(pr)
            elif mode == 'probs':
                grid[i,j]=np.max(pr)

    return grid_x, grid_y, grid
  

def plot_decision_boundary(ax1, u, v, z, levels=None, colors=['k','k','k']):
    # Basic contour plot
#     fig1, ax1 = plt.subplots()
    if levels == None:
        ctp = ax1.contour(u, v, z, alpha=0.6)
    else:
        ctp = ax1.contour(u, v, z, levels=levels, colors=colors, alpha=0.6)
        # Description of contours
        fmt = {l: str(l) for l in ctp.levels}
        ax1.clabel(ctp, ctp.levels, inline=True, fmt=fmt, fontsize=10)
        ax1.contourf(u, v, z, cmap=plt.cm.Paired)
    return ax1

In [61]:
@interact(num_epochs=(0,10000,500), 
          learning_rate=[0.01,0.02,0.03], 
          c1_samples='20',
          c1_scatter=(1,8,0.4),
          c2_samples='20',
          c2_scatter=(1,8,0.4),
          c3_samples='20',
          c3_scatter=(1,8,0.4),)
def multi_log_regression(num_epochs, 
                         learning_rate, 
                         c1_samples,
                         c1_scatter,
                         c2_samples, 
                         c2_scatter,
                         c3_samples,
                         c3_scatter,):
    if  c1_samples.isdigit()==False or c2_samples.isdigit()==False or c2_samples.isdigit()==False:
            pass
    else:
        c1_samples = int(c1_samples)
        c2_samples = int(c2_samples)
        c3_samples = int(c3_samples)
        x, labels, samples = make_data((c1_samples, c2_samples, c3_samples),
                                       (c1_scatter, c2_scatter, c3_scatter))
#         w = 0.001 * np.ones((3,x.shape[1])) # #classes x features
#         for epoch in range(num_epochs):
#                 loss, dw, probs = loss_fn(x,w,labels)
#                 acc = np.sum(np.argmax(labels)==np.argmax(probs))
#                 w -= learning_rate*dw
#         u,v,z = grid(x,w,mode='sep')
#         acc = np.sum(np.argmax(labels, axis=1)==np.argmax(probs, axis=1))
#         acc /= x.shape[0]
        
        
        
#         ax = plot_data(*samples)
        
#         # Plot missclassified data
#         if acc != 1.0:
#             idx = np.argmax(labels, axis=1)!=np.argmax(probs, axis=1)
#             ax.scatter(x[idx,1], x[idx,2], marker='x', c='k')
        
        
#         try: 
#             ax.text(9,10,'loss: {0:.4f}'.format(loss))
#             ax.text(9,11,'acc: {0:.4f}'.format(acc)) 
#         except:
#             pass

#         ax = plot_decision_boundary(ax, u,v,z.T)

        
#         plt.xlim(-4,14)
#         plt.ylim(-4,14)
#         ax.figure
    for multi_class in (['multinomial']):
        clf = LogisticRegression(solver='sag', max_iter=num_epochs, random_state=42,
                                 multi_class=multi_class).fit(x, np.argmax(labels, axis=1))

        # print the training scores
        #print("training score : %.3f (%s)" % (clf.score(x, np.argmax(labels, axis=1)), multi_class))

        # create a mesh to plot in
        h = .02  # step size in the mesh
        x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
        y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
        # Put the result into a color plot
        print(Z[40:70])
        Z = Z.reshape(xx.shape)

        plt.figure()
        plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

        ids = clf.predict(x) != np.argmax(labels, axis=1)
        plt.scatter(x[ids,0], x[ids,1], marker='x', c='k')
        # Plot also the training points
        colors = "bry"
        for i, color in zip(clf.classes_, colors):
            idx = np.where(np.argmax(labels, axis=1) == i)
            plt.scatter(x[idx, 0], x[idx, 1], c=color, cmap=plt.cm.Paired,
                        edgecolor='black', s=20)
        
#     return ax
    

interactive(children=(IntSlider(value=5000, description='num_epochs', max=10000, step=500), Dropdown(descripti…

In [62]:
@interact(num_epochs=(0,10000,500), 
          learning_rate=[0.01,0.02,0.03], 
          c1_samples='20',
          c1_scatter=(1,8,0.4),
          c2_samples='20',
          c2_scatter=(1,8,0.4),
          c3_samples='20',
          c3_scatter=(1,8,0.4),)
def multi_log_regression(num_epochs, 
                         learning_rate, 
                         c1_samples,
                         c1_scatter,
                         c2_samples, 
                         c2_scatter,
                         c3_samples,
                         c3_scatter,):
    if  c1_samples.isdigit()==False or c2_samples.isdigit()==False or c2_samples.isdigit()==False:
            pass
    else:
        c1_samples = int(c1_samples)
        c2_samples = int(c2_samples)
        c3_samples = int(c3_samples)
        x, labels, samples = make_data((c1_samples, c2_samples, c3_samples),
                                       (c1_scatter, c2_scatter, c3_scatter))
        
#         # Denis
        w = 0.001 * np.ones((3,x.shape[1])) # #classes x features
        for epoch in range(num_epochs):
                loss, dw, probs = loss_fn(x,w,labels)
                acc = np.sum(np.argmax(labels)==np.argmax(probs))
                w -= learning_rate*dw
#         u,v,z = grid(x,w,mode='sep')

        
    # SK
    
    clf = LogisticRegression(solver='sag', max_iter=num_epochs, random_state=42,
                             multi_class='multinomial').fit(x, np.argmax(labels, axis=1))
    print(probs)

#     w = np.hstack((clf.intercept_[:,None],clf.coef_))
    u,v,z = grid(x,w,mode='sep')
    ax = plot_data(*samples)


    try: 
        ax.text(9,10,'loss: {0:.4f}'.format(loss))
        ax.text(9,11,'acc: {0:.4f}'.format(acc)) 
    except:
        pass

    ax = plot_decision_boundary(ax, u,v,z.T)


    plt.xlim(-8,12)
    plt.ylim(-8,15)


    ax.figure


interactive(children=(IntSlider(value=5000, description='num_epochs', max=10000, step=500), Dropdown(descripti…

In [50]:

# from sklearn.linear_model import LogisticRegression


# for multi_class in ('multinomial', 'ovr'):
#     clf = LogisticRegression(solver='sag', max_iter=100, random_state=42,
#                              multi_class=multi_class).fit(x, labels)

#     # print the training scores
#     print("training score : %.3f (%s)" % (clf.score(x, labels), multi_class))

#     # create a mesh to plot in
#     h = .02  # step size in the mesh
#     x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
#     y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
#     xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
#                          np.arange(y_min, y_max, h))

#     # Plot the decision boundary. For that, we will assign a color to each
#     # point in the mesh [x_min, x_max]x[y_min, y_max].
#     Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
#     # Put the result into a color plot
#     Z = Z.reshape(xx.shape)
#     plt.figure()
#     plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
#     plt.title("Decision surface of LogisticRegression (%s)" % multi_class)
#     plt.axis('tight')

#     # Plot also the training points
#     colors = "bry"
#     for i, color in zip(clf.classes_, colors):
#         idx = np.where(y == i)
#         plt.scatter(x[idx, 0], x[idx, 1], c=color, cmap=plt.cm.Paired,
#                     edgecolor='black', s=20)

#     # Plot the three one-against-all classifiers
#     xmin, xmax = plt.xlim()
#     ymin, ymax = plt.ylim()
#     coef = clf.coef_
#     intercept = clf.intercept_

#     def plot_hyperplane(c, color):
#         def line(x0):
#             return (-(x0 * coef[c, 0]) - intercept[c]) / coef[c, 1]
#         plt.plot([xmin, xmax], [line(xmin), line(xmax)],
#                  ls="--", color=color)

#     for i, color in zip(clf.classes_, colors):
#         plot_hyperplane(i, color)

# plt.show()

In [6]:
np.c_()

TypeError: 'CClass' object is not callable